In [2]:
import os
import cv2
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

2023-09-29 22:38:30.534740: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
with open('../data/external/data.pkl','rb') as f:
    X_train,y_train,X_test,y_test = pickle.load(f)

In [4]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (725, 1080, 1080, 3)
y_train shape: (725,)
X_test shape: (182, 1080, 1080, 3)
y_test shape: (182,)


In [5]:
# Define the data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest'
)

In [6]:
# Apply data augmentation to the training set
X_train_augmented = []
y_train_augmented = []

for i in range(X_train.shape[0]):
    img = X_train[i]
    label = y_train[i]
    
    # Expand the dimensions of the image to (1, 1080, 1080, 3)
    img = np.expand_dims(img, axis=0)
    
    for j in range(2):
        x_augmented = datagen.flow(img, batch_size=1).next()
        X_train_augmented.append(x_augmented[0])  # Extract the augmented image
        y_train_augmented.append(label)


In [7]:
# Convert the augmented data to NumPy arrays
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

: 

In [6]:
# Apply data augmentation to the test set
X_test_augmented = []
y_test_augmented = []

for i in range(X_test.shape[0]):
    img = X_test[i]
    label = y_test[i]
    
    # Expand the dimensions of the image to (1, 1080, 1080, 3)
    img = np.expand_dims(img, axis=0)
    
    for j in range(2):
        x_augmented = datagen.flow(img, batch_size=1).next()
        X_test_augmented.append(x_augmented[0])  # Extract the augmented image
        y_test_augmented.append(label)

In [ ]:
X_test_augmented = np.array(X_test_augmented)
y_test_augmented = np.array(y_test_augmented)

In [ ]:
print(f'X_train_augmented shape: {X_train_augmented.shape}')
print(f'y_train_augmented shape: {y_train_augmented.shape}')
print(f'X_test_augmented shape: {X_test_augmented.shape}')
print(f'y_test_augmented shape: {y_test_augmented.shape}')

In [ ]:
data = (X_train_augmented,y_train_augmented,X_test_augmented,y_test_augmented)
with open('../data/external/Augmended_data.pkl','wb') as f:
    pickle.dump(Augmented_data, f)